In [1]:
from pyspark.sql.functions import max, min, sum, mean, when

# Run the date count jobs tasks
from jobs.solana import SparkJob

job = SparkJob()
df = job.solana_sql_udf()


preprocessing completed.
+----+--------+
|open|neg_open|
+----+--------+
|   2|      -2|
|   3|      -3|
|   3|      -3|
|   3|      -3|
|   2|      -2|
|   3|      -3|
|   3|      -3|
|   2|      -2|
|   2|      -2|
|   2|      -2|
+----+--------+
only showing top 10 rows

